##Overall Architecture used for the LCP shared Task

* Contains the individual ML Models used

Note
* The results in the output of the cells are the best results obtained using the checkpoints saved at the time of training. 
* Slight deviations are possible if everything is run from scratch due to certain non-determinism in PyTorch computations

In [1]:
!pip install spacy-syllables
!python -m spacy download en_core_web_sm
!pip3 install wordfreq

     |████████████████████████████████| 1.9MB 16.8MB/s 
     |████████████████████████████████| 12.8MB 292kB/s 
     |████████████████████████████████| 460kB 49.9MB/s 
     |████████████████████████████████| 1.1MB 32.9MB/s 
     |████████████████████████████████| 9.1MB 33.6MB/s 
     |████████████████████████████████| 122kB 48.7MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=202ac3854f626823ec3c1b02fe983afb06633d0f78466b7f2d145f8f40aa37f3
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: smart-open 4.2.0
    Uninstalling smart-open-4.2.0:
      Successfully uninstalled smart-open-4.2.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsl

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from wordfreq import word_frequency
from scipy import stats
import csv
import spacy
from spacy_syllables import SpacySyllables
import random
import os

In [ ]:
# We saved certain models while training (biLSTM to predict context probability) to avoid re-training again

# Steps To use already trained models
# 1. Uncomment the below cell two lines
# 2. Download the TrainedModels folder in the repo
# 3. Upload the whole folder to your google drive
# 4. Run the below cell and authenticate the Google Drive request
# 5. The cells later where trained and saved models can be used have instructions on how to use them

In [23]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-04-10 08:07:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-10 08:07:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-10 08:07:28--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
!unzip glove*.zip
!ls
!pwd

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data
/content


In [6]:
# https://www.kaggle.com/bminixhofer/deterministic-neural-networks-using-pytorch
# Seed all rngs for deterministic results
def seed_all(seed = 0):
  random.seed(0)
  os.environ['PYTHONHASHSEED'] = str(seed)
  torch.manual_seed(seed)
  np.random.seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

In [7]:
seed_all(0)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("syllables", after='tagger') # Add the syllable tagger pipe

In [10]:
SINGLE_TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
SINGLE_TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv"

MULTI_TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_multi_train.tsv"
MULTI_TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_multi_test.tsv"

In [11]:
def get_data_frames():
  df_train_single = pd.read_csv(SINGLE_TRAIN_DATAPATH, sep='\t', quotechar="'", quoting=csv.QUOTE_NONE)
  df_test_single = pd.read_csv(SINGLE_TEST_DATAPATH, sep='\t', quotechar="'", quoting=csv.QUOTE_NONE)

  df_train_multi = pd.read_csv(MULTI_TRAIN_DATAPATH, sep='\t', quotechar="'", quoting=csv.QUOTE_NONE)
  df_test_multi = pd.read_csv(MULTI_TEST_DATAPATH, sep='\t', quotechar="'", quoting=csv.QUOTE_NONE)

  return df_train_single, df_test_single, df_train_multi, df_test_multi

In [12]:
df_train_single, df_test_single, df_train_multi, df_test_multi = get_data_frames()

Features used 

* Word Embedding [GloVe 50 dimensional embeddings](http://nlp.stanford.edu/data/glove.6B.zip)
* Length of word
* Syllable count [PyPy](https://pypi.org/project/syllables/)
* Word Frequency [PyPy](https://pypi.org/project/wordfreq/)
* POS tag [Spacy](https://spacy.io/usage/linguistic-features#pos-tagging)

[Reference](https://www.aclweb.org/anthology/W18-0508.pdf)


In [13]:
single_tokens_train_raw = df_train_single["token"].astype(str).to_list()
single_tokens_test_raw = df_test_single["token"].astype(str).to_list()

y_single_train = df_train_single["complexity"].astype(np.float32).to_numpy()
y_single_test = df_test_single["complexity"].astype(np.float32).to_numpy()

multi_tokens_train_raw = df_train_multi["token"].astype(str).to_list()
multi_tokens_test_raw = df_test_multi["token"].astype(str).to_list()

y_multi_train = df_train_multi["complexity"].astype(np.float32).to_numpy()
y_multi_test = df_test_multi["complexity"].astype(np.float32).to_numpy()

sent_train_single_raw = df_train_single["sentence"].to_list()
sent_test_single_raw = df_test_single["sentence"].to_list()

sent_train_multi_raw = df_train_multi["sentence"].to_list()
sent_test_multi_raw = df_test_multi["sentence"].to_list()

In [14]:
EMBEDDING_DIM = 50

def get_embeddings():
  embedding_index = {}
  with open('glove.6B.{}d.txt'.format(EMBEDDING_DIM), 'r', encoding='utf-8') as f:
    for line in f:
      values = line.split()
      token = values[0]
      embedding_index[token] = np.asarray(values[1:], dtype='float32')
  return embedding_index

In [15]:
embedding_index = get_embeddings()
print('Token count in embeddings: {}'.format(len(embedding_index)))

Token count in embeddings: 400000


biLSTM to predict target probability

Reference - [PyTorch](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)


In [16]:
HIDDEN_DIM = 10

In [17]:
def prepare_sequence(seq, to_ix):
  seq = seq.split()
  idxs = [to_ix[w.lower()] if w.lower() in to_ix else len(to_ix) for w in seq]
  idxs = torch.tensor(idxs)
  idxs = nn.functional.one_hot(idxs, num_classes=len(to_ix))
  idxs = torch.tensor(idxs, dtype=torch.float32)
  return idxs


def map_token_to_idx():
  word_to_ix = {}
  word_to_ix_multi = {}
  for sent in sent_train_single_raw:
    sent = sent.split()
    for word in sent:
      word = word.lower()
      if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)

  for sent in sent_train_multi_raw:
    sent = sent.split()
    for word in sent:
      word = word.lower()
      if word not in word_to_ix_multi:
        word_to_ix_multi[word] = len(word_to_ix_multi)
  
  return word_to_ix, word_to_ix_multi

In [18]:
word_to_ix, word_to_ix_multi = map_token_to_idx()
print('SWE vocab size: {}\nMWE vocab size: {}'.format(len(word_to_ix), len(word_to_ix_multi)))

SWE vocab size: 24350
MWE vocab size: 9699


biLSTM class to calculate token probability given context

In [19]:
class biLSTM(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
    super(biLSTM, self).__init__()
    self.hidden_dim = hidden_dim
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
    self.hidden2tag = nn.Linear(2 * hidden_dim, output_size)

  def prepare_embedding(self, sentence):
    embeddings = []
    for word in sentence:
      word = word.lower()
      if word in embedding_index:
        embeddings.extend(embedding_index[word])
      else:
        embeddings.extend(np.random.random(EMBEDDING_DIM).tolist())
    embeddings = torch.tensor(embeddings, dtype=torch.float32, device=device)
    return embeddings

  def forward(self, sentence):
    sentence = sentence.split()
    embeds = self.prepare_embedding(sentence)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.softmax(tag_space, dim=1)
    return tag_scores

biLSTM model for single word targets

In [20]:
model = biLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(word_to_ix))
# Used while training phase to avoid re training the same model again and again
# To use the saved model, change the False in next cell to True (Ensure you've uploaded the model in your Google Drive using steps mentioned earlier)
path_biLSTM_single = '/content/gdrive/My Drive/TrainedModels/biLSTM.pt'

In [21]:
USE_PRETRAINED_SINGLE_WORD_TARGET_MODEL = False

In [24]:
if USE_PRETRAINED_SINGLE_WORD_TARGET_MODEL:
  print('Using pre-trained biLSTM on single target expressions')
  model = torch.load(path_biLSTM_single)
  model.eval()
else:
  print('Training biLSTM on single target expressions')
  # Train the model for 10 epochs
  model = biLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(word_to_ix))
  loss_function = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01)
  for epoch in range(10):
    loss_sum = 0
    for sentence in sent_train_single_raw:
      model.zero_grad()
      targets = prepare_sequence(sentence, word_to_ix)
      tag_scores = model(sentence)
      loss = loss_function(tag_scores, targets)
      loss_sum += loss
      loss.backward()
      optimizer.step()
    print('Epoch: {} Loss: {}'.format(epoch, loss_sum.item()))
  # path_biLSTM_single = '/content/gdrive/My Drive/TrainedModels/biLSTM.pt'
  # torch.save(model, path_biLSTM_single)

Using pre-trained biLSTM on single target expressions


biLSTM model for multi word targets

In [25]:
model_multi = biLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix_multi), len(word_to_ix_multi))
# Used while training phase to avoid re training the same model again and again
# To use the saved model, change the False in next cell to True (Ensure you've uploaded the model in your Google Drive using steps mentioned earlier)
path_biLSTM_multi = '/content/gdrive/My Drive/TrainedModels/biLSTM_multi.pt'

In [26]:
USE_PRETRAINED_MULTI_WORD_TARGET_MODEL = False

In [27]:
if USE_PRETRAINED_MULTI_WORD_TARGET_MODEL:
  print('Using pre-trained biLSTM on multi target expressions')
  model_multi = torch.load(path_biLSTM_multi)
  model_multi.eval()
else:
  print('Training biLSTM on multi target expressions')
  model_multi = biLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix_multi), len(word_to_ix_multi))
  loss_function = nn.MSELoss()
  optimizer = optim.Adam(model_multi.parameters(), lr=0.01)
  for epoch in range(10):
    loss_sum = 0
    for sentence in sent_train_multi_raw:
      model_multi.zero_grad()
      targets = prepare_sequence(sentence, word_to_ix_multi)
      tag_scores = model_multi(sentence)
      loss = loss_function(tag_scores, targets)
      loss_sum += loss
      loss.backward()
      optimizer.step()
    print('Epoch: {} Loss: {}'.format(epoch, loss_sum.item()))
  # path_biLSTM_multi = '/content/gdrive/My Drive/TrainedModels/biLSTM_multi.pt'
  # torch.save(model_multi, path_biLSTM_multi)

Using pre-trained biLSTM on multi target expressions


In [28]:
def prepare_features_single_word(tokens, sentences):
  features = []
  for idx, word in enumerate(tokens):
    word = word.lower()
    feature = []

    # Word length
    feature.append(len(word))
    doc = nlp(word)

    # Syllable count and word frequency in the corpus
    # Spacy tokenizes the input sentence
    # In this case we would have only one token, the target word
    for token in doc:
      feature.append(token._.syllables_count)
      feature.append(word_frequency(word, 'en'))

    # Probability of target word `word` in the sentence estimated from by `model`
    if word in word_to_ix:
      # Output scores for each of the word in the sentence
      out = model(sentences[idx])
      pos = -1
      for itr, token in enumerate(sentences[idx].split()):
        if token.lower() == word:
          pos = itr
          break
      id_pos = word_to_ix[word] # word to id mapping
      feature.append(float(out[pos][id_pos]))
    else:
      # `word` not in vocabulary, so cannot predict probability in context
      feature.append(0.0)

    # GloVE embedding for the `word`
    if word in embedding_index:
      feature.extend(embedding_index[word].tolist())
    else:
      # `word` not in the GloVE corpus, take a random embedding
      feature.extend(np.random.random(EMBEDDING_DIM).tolist())
    features.append(feature)

    if (idx + 1) % 500 == 0:
      print('Prepared features for {} single target word sentences'.format(idx + 1))
  return features

In [29]:
def prepare_features_multi_word(tokens, sentences):
  features = []
  for idx, word in enumerate(tokens):
    word = word.lower()
    feature = []
    doc = nlp(word)
    word = word.split(' ')
    assert(len(word) == 2)

    # MWE length = sum(length of individual words)
    feature.append(len(word[0]) + len(word[1]))

    syllables = 0
    probability = 1
    embedding = np.zeros(EMBEDDING_DIM)

    # Syllable count and word frequency in the corpus
    # Spacy tokenizes the input sentence
    # In this case we would have two tokens

    for token in doc:
      word_ = token.text
      syllables += token._.syllables_count
      probability *= word_frequency(word_, 'en')

      # GloVE embedding current `word_` of the MWE
      if word_ in embedding_index:
        embedding = embedding + embedding_index[word_]
      else:
        # `word_` not in the GloVE corpus, take a random embedding
        embedding = embedding + np.random.random(EMBEDDING_DIM)

    # Average embedding of the two tokens in the MWE
    embedding = embedding / 2
    feature.append(syllables)
    feature.append(probability)

    # Product of probabilities of constituent words in the MWE
    if word[0] in word_to_ix_multi and word[1] in word_to_ix_multi:
      # Output scores for each of the word in the sentence
      out = model_multi(sentences[idx])
      pos0, pos1 = -1, -1
      for itr, token in enumerate(sentences[idx].split()):
        if token.lower() == word[0]:
          pos0 = itr
          pos1 = itr + 1
          break
      id_pos0 = word_to_ix_multi[word[0]]
      id_pos1 = word_to_ix_multi[word[1]]
      feature.append(float(out[pos0][id_pos0] * out[pos1][id_pos1]))
    else:
      # Either of the constituent words of the MWE not in vocabulary \
      # So cannot predict probability in context
      feature.append(0.0)

    feature.extend(embedding.tolist())
    features.append(feature)

    if (idx + 1) % 500 == 0:
      print('Prepared features for {} multi target word sentences'.format(idx + 1))

  return features

In [30]:
print('+++ Generating Train features for Single word expressions +++')
features_train_single = prepare_features_single_word(single_tokens_train_raw, sent_train_single_raw)
print('+++ Generating Test features for Single word expressions +++')
features_test_single = prepare_features_single_word(single_tokens_test_raw, sent_test_single_raw)

+++ Generating Train features for Single word expressions +++
Prepared features for 500 single target word sentences
Prepared features for 1000 single target word sentences
Prepared features for 1500 single target word sentences
Prepared features for 2000 single target word sentences
Prepared features for 2500 single target word sentences
Prepared features for 3000 single target word sentences
Prepared features for 3500 single target word sentences
Prepared features for 4000 single target word sentences
Prepared features for 4500 single target word sentences
Prepared features for 5000 single target word sentences
Prepared features for 5500 single target word sentences
Prepared features for 6000 single target word sentences
Prepared features for 6500 single target word sentences
Prepared features for 7000 single target word sentences
Prepared features for 7500 single target word sentences
+++ Generating Test features for Single word expressions +++
Prepared features for 500 single targe

In [31]:
print('+++ Generating Train features for Multi word expressions +++')
features_train_multi = prepare_features_multi_word(multi_tokens_train_raw, sent_train_multi_raw)
print('+++ Generating Test features for Multi word expressions +++')
features_test_multi = prepare_features_multi_word(multi_tokens_test_raw, sent_test_multi_raw)

+++ Generating Train features for Multi word expressions +++
Prepared features for 500 multi target word sentences
Prepared features for 1000 multi target word sentences
Prepared features for 1500 multi target word sentences
+++ Generating Test features for Multi word expressions +++


In [32]:
# Convert all features to torch.tensor to enable use in PyTorch models
X_train_single_tensor = torch.tensor(features_train_single, dtype=torch.float32, device=device)
X_test_single_tensor = torch.tensor(features_test_single, dtype=torch.float32, device=device)
X_train_multi_tensor = torch.tensor(features_train_multi, dtype=torch.float32, device=device)
X_test_multi_tensor = torch.tensor(features_test_multi, dtype=torch.float32, device=device)

In [33]:
# Reshape all output complexity scores to single dimension vectors
y_single_train = y_single_train.reshape(y_single_train.shape[0], -1)
y_single_test = y_single_test.reshape(y_single_test.shape[0], -1)
y_multi_train = y_multi_train.reshape(y_multi_train.shape[0], -1)
y_multi_test = y_multi_test.reshape(y_multi_test.shape[0], -1)

In [34]:
# Convert all target outputs to torch.tensor to enable use in PyTorch models
Y_train_single_tensor = torch.tensor(y_single_train, dtype=torch.float32, device=device)
Y_test_single_tensor = torch.tensor(y_single_test, dtype=torch.float32, device=device)
Y_train_multi_tensor = torch.tensor(y_multi_train, dtype=torch.float32, device=device)
Y_test_multi_tensor = torch.tensor(y_multi_test, dtype=torch.float32, device=device)

In [35]:
# Ensure each sample from test and train for single word expression is taken
print(X_train_single_tensor.shape)
print(X_test_single_tensor.shape)
print(Y_train_single_tensor.shape)
print(Y_test_single_tensor.shape)

torch.Size([7662, 54])
torch.Size([917, 54])
torch.Size([7662, 1])
torch.Size([917, 1])


In [36]:
# Ensure each sample from test and train for multi word expression is taken
print(X_train_multi_tensor.shape)
print(X_test_multi_tensor.shape)
print(Y_train_multi_tensor.shape)
print(Y_test_multi_tensor.shape)

torch.Size([1517, 54])
torch.Size([184, 54])
torch.Size([1517, 1])
torch.Size([184, 1])


In [37]:
def convert_tensor_to_np(y):
  if device == torch.device("cuda"):
    y = y.cpu()
  y = y.detach().numpy()
  return y

In [38]:
from copy import deepcopy

In [39]:
# Evaluate the metrics upon which the model would be evaluated
def evaluate_metrics(labels, predicted):
  vx, vy = [], []
  if torch.is_tensor(labels):
    vx = labels.clone()
    vx = convert_tensor_to_np(vx)
  else:
    vx = deepcopy(labels)
  if torch.is_tensor(predicted):
    vy = predicted.clone()
    vy = convert_tensor_to_np(vy)
  else:
    vy = deepcopy(predicted)

  pearsonR = np.corrcoef(vx.T, vy.T)[0, 1]
  spearmanRho = stats.spearmanr(vx, vy)
  MSE = np.mean((vx - vy) ** 2)
  MAE = np.mean(np.absolute(vx - vy))
  RSquared = pearsonR ** 2

  print("Peason's R: {}".format(pearsonR))
  print("Spearman's rho: {}".format(spearmanRho))
  print("R Squared: {}".format(RSquared))
  print("MSE: {}".format(MSE))
  print("MAE: {}".format(MAE))

## Neural Network

* $N$ input sentences  

* d (=EMBEDDING_DIM) word embedding

* $I$ = Word Embedding matrix ($N \times d$)

* $W_1, W_2, W_3, W_4 := (d \times 256), (256 \times 128), (128 \times 64), (64 \times 1)$

* Equations

  * $o_1 = tanh(I \times W_1 + b_1)$

  * $o_2 = tanh(o_1 \times W_2 + b_2)$

  * $o_3 = tanh(o_2 \times W_3 + b_3)$

  * $o_4 = \sigma(o_3 \times W_4)$


In [40]:
class NN(nn.Module):
  def __init__(self, embedding_dim):
    super(NN, self).__init__()
    self.linear1 = nn.Linear(embedding_dim, 128, bias=True)
    self.linear2 = nn.Linear(128, 256, bias=True)
    self.linear3 = nn.Linear(256, 64, bias=True)
    self.linear4 = nn.Linear(64, 1)

  def forward(self, input):
    out = torch.tanh(self.linear1(input))
    out = torch.tanh(self.linear2(out))
    out = torch.tanh(self.linear3(out))
    out = torch.sigmoid(self.linear4(out))
    return out

In [41]:
loss_function = nn.MSELoss()

In [42]:
embedding_dim = X_train_single_tensor.shape[1]
model_NN = NN(embedding_dim)
model_NN.to(device)
# Used while training phase to save the best checkpoint (with the best Pearson R on test set)
# To use the saved model, change the False in next cell to True (Ensure you've uploaded the model in your Google Drive using steps mentioned earlier)
path_NN = '/content/gdrive/My Drive/TrainedModels/NN_0.731.pt'

In [43]:
USE_PRETRAINED_SINGLE_WORD_TARGET_NN = False

In [44]:
if USE_PRETRAINED_SINGLE_WORD_TARGET_NN:
  print('Using pre-trained NN on single target expressions')
  model_NN = torch.load(path_NN)
  model_NN.eval()
else:
  print('Training NN on single target expressions...')
  model_NN = NN(embedding_dim)
  model_NN.to(device)
  loss_function = nn.MSELoss()
  optimizer = optim.Adam(model_NN.parameters(), lr=0.002)
  for epoch in range(30):
    optimizer.zero_grad()
    out = model_NN(X_train_single_tensor)
    loss = loss_function(out, Y_train_single_tensor)
    loss.backward()
    optimizer.step()
    print("Epoch {} : {}".format(epoch + 1, loss.item()))
  # path_NN = '/content/gdrive/My Drive/TrainedModels/NN_0.731.pt'
  # torch.save(model_NN, path_NN)  

Using pre-trained NN on single target expressions


In [45]:
out_NN = model_NN(X_test_single_tensor)
evaluate_metrics(out_NN, Y_test_single_tensor)

Peason's R: 0.73164611615449
Spearman's rho: SpearmanrResult(correlation=0.7083250660997157, pvalue=1.454658192615246e-140)
R Squared: 0.5353060392839495
MSE: 0.00759829580783844
MAE: 0.06738824397325516


In [46]:
out_NN[0]

tensor([0.1671], grad_fn=<SelectBackward>)

In [47]:
embedding_dim = X_train_multi_tensor.shape[1]
model_NN_multi = NN(embedding_dim)
model_NN_multi.to(device)
# Used while training phase to save the best checkpoint (with the best Pearson R on test set)
# To use the saved model, change the False in next cell to True (Ensure you've uploaded the model in your Google Drive using steps mentioned earlier)
path_NN_multi = '/content/gdrive/My Drive/TrainedModels/NN_multi_0.775.pt'

In [48]:
USE_PRETRAINED_MULTI_WORD_TARGET_NN = False

In [49]:
if USE_PRETRAINED_MULTI_WORD_TARGET_NN:
  print('Using pre-trained NN on multi target expressions')
  model_NN_multi = torch.load(path_NN_multi)
  model_NN_multi.eval()
else:
  print('Training NN on multi target expressions...')
  model_NN_multi = NN(embedding_dim)
  model_NN_multi.to(device)
  loss_function = nn.MSELoss()
  optimizer = optim.Adam(model_NN_multi.parameters(), lr=0.002)
  for epoch in range(30):
    optimizer.zero_grad()
    out = model_NN_multi(X_train_multi_tensor)
    loss = loss_function(out, Y_train_multi_tensor)
    loss.backward()
    optimizer.step()
    print("Epoch {} : {}".format(epoch + 1, loss.item()))
  # path_NN_multi = '/content/gdrive/My Drive/TrainedModels/NN_multi_0.774.pt'
  # torch.save(model_NN_multi, path_NN_multi)

Using pre-trained NN on multi target expressions


In [50]:
out_NN_multi = model_NN_multi(X_test_multi_tensor)
evaluate_metrics(out_NN_multi, Y_test_multi_tensor)

Peason's R: 0.775311152864301
Spearman's rho: SpearmanrResult(correlation=0.7682863746456144, pvalue=4.1883306755893856e-37)
R Squared: 0.6011073837557714
MSE: 0.010189074091613293
MAE: 0.07908465713262558


## Machine Learning Methods

* Linear Regression

* Support Vector Regressor

In [51]:
X_train_single_np = np.array(features_train_single)
X_test_single_np = np.array(features_test_single)
Y_train_single_np = np.array(y_single_train.reshape(y_single_train.shape[0], -1))
Y_test_single_np = np.array(y_single_test.reshape(y_single_test.shape[0], -1))

In [52]:
print(X_train_single_np.shape)
print(X_test_single_np.shape)
print(Y_train_single_np.shape)
print(Y_test_single_np.shape)

(7662, 54)
(917, 54)
(7662, 1)
(917, 1)


In [53]:
X_train_multi_np = np.array(features_train_multi)
X_test_multi_np = np.array(features_test_multi)
Y_train_multi_np = np.array(y_multi_train.reshape(y_multi_train.shape[0], -1))
Y_test_multi_np = np.array(y_multi_test.reshape(y_multi_test.shape[0], -1))

In [54]:
print(X_train_multi_np.shape)
print(X_test_multi_np.shape)
print(Y_train_multi_np.shape)
print(Y_test_multi_np.shape)

(1517, 54)
(184, 54)
(1517, 1)
(184, 1)


### Linear Regression

In [55]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [56]:
from sklearn.linear_model import LinearRegression

In [57]:
def evaluateLinearRegression(X_train, Y_train, X_test, Y_test):
  reg = make_pipeline(StandardScaler(), LinearRegression())
  reg.fit(X_train, Y_train)
  out = reg.predict(X_test)
  out = out.reshape((out.shape[0], 1))
  evaluate_metrics(out, Y_test)
  return out

In [58]:
print('Linear Regression for Single word expressions')
out_LR = evaluateLinearRegression(X_train_single_np, Y_train_single_np, X_test_single_np, Y_test_single_np)

Linear Regression for Single word expressions
Peason's R: 0.7108434116187733
Spearman's rho: SpearmanrResult(correlation=0.6918562248147357, pvalue=1.4505768829498638e-131)
R Squared: 0.5052983558418167
MSE: 0.00801003205435963
MAE: 0.06855075784165898


In [59]:
print('Linear Regression for Multi word expressions')
out_LR_multi = evaluateLinearRegression(X_train_multi_np, Y_train_multi_np, X_test_multi_np, Y_test_multi_np)

Linear Regression for Multi word expressions
Peason's R: 0.7672485716855574
Spearman's rho: SpearmanrResult(correlation=0.7573872177422218, pvalue=1.5861218509138286e-35)
R Squared: 0.5886703707535279
MSE: 0.00994057080022215
MAE: 0.08001505070711726


### Support Vector Regressor

* Radial basis function
* C = 0.05
* epsilon = 0.01

In [60]:
from sklearn.svm import SVR

In [61]:
def evaluateSVR(X_train, Y_train, X_test, Y_test):
  svr = make_pipeline(StandardScaler(), SVR(C=0.05, epsilon=0.01))
  svr.fit(X_train, Y_train.reshape(-1))
  out = svr.predict(X_test)
  out = out.reshape((out.shape[0], 1))
  evaluate_metrics(out, Y_test)
  return out

In [62]:
print('SVR for Single word expressions')
out_svr = evaluateSVR(X_train_single_np, Y_train_single_np, X_test_single_np, Y_test_single_np)

SVR for Single word expressions
Peason's R: 0.732967844251433
Spearman's rho: SpearmanrResult(correlation=0.7037429683169284, pvalue=5.357912593044118e-138)
R Squared: 0.537241860706593
MSE: 0.007531175064386141
MAE: 0.06719389225485689


In [63]:
print('SVR for Multi word expressions')
out_svr_multi = evaluateSVR(X_train_multi_np, Y_train_multi_np, X_test_multi_np, Y_test_multi_np)

SVR for Multi word expressions
Peason's R: 0.7834043625882116
Spearman's rho: SpearmanrResult(correlation=0.754343227255628, pvalue=4.2297035272986037e-35)
R Squared: 0.613722395322242
MSE: 0.010053853343430836
MAE: 0.0810940654531415


### Summary 


| Model | Type | Pearson | MSE | MAE|
|---|---|---|---|---|
|Neural Network| Single word| 0.7315 | 0.0077 | 0.0681|
|Neural Network| Multi word| 0.7753 | 0.0100 | 0.0780|
|Linear Regression| Single word| 0.7108 | 0.0080 | 0.0685|
|Linear Regression| Multi word| 0.7672 | 0.0099 | 0.0800|
|SVR| Single word| 0.7330 | 0.0075 | 0.0672|
|SVR| Multi word| 0.7834 | 0.0100 | 0.0811|



In [64]:
single_ids = df_test_single["id"].astype(str).to_list()
multi_ids = df_test_multi["id"].astype(str).to_list()

Aggregation of results obtained

In [65]:
out_ensemble = []

for idx in range(len(out_NN)):
  score = 0
  score += float(out_NN[idx])
  score += float(out_LR[idx])
  score += float(out_svr[idx])
  if idx == 0:
    print(float(out_NN[idx]), float(out_LR[idx]), float(out_svr[idx]), score / 3)
  score /= 3
  out_ensemble.append(score)
out_ensemble = np.array(out_ensemble)
out_ensemble = out_ensemble.reshape((out_ensemble.shape[0], 1))

0.16710498929023743 0.1179137668165319 0.11636303710497864 0.13379393107058266


In [66]:
evaluate_metrics(out_ensemble, Y_test_single_np)

Peason's R: 0.7381368031461403
Spearman's rho: SpearmanrResult(correlation=0.7105119840090347, pvalue=8.322838188474213e-142)
R Squared: 0.5448459401588038
MSE: 0.007398631813890535
MAE: 0.06638338984666789


In [67]:
out_ensemble_multi = []

for idx in range(len(out_NN_multi)):
  score = 0
  score += float(out_NN_multi[idx])
  score += float(out_LR_multi[idx])
  score += float(out_svr_multi[idx])
  score /= 3
  out_ensemble_multi.append(score)
out_ensemble_multi = np.array(out_ensemble_multi)
out_ensemble_multi = out_ensemble_multi.reshape((out_ensemble_multi.shape[0], 1))

In [68]:
evaluate_metrics(out_ensemble_multi, Y_test_multi_np)

Peason's R: 0.7881390769343021
Spearman's rho: SpearmanrResult(correlation=0.7706449522570928, pvalue=1.8581243837015525e-37)
R Squared: 0.6211632045908537
MSE: 0.009589619324499697
MAE: 0.07759087966511614


In [69]:
# path_out_single = '/content/gdrive/My Drive/NLP_Outputs/out_single.csv'
# path_out_multi = '/content/gdrive/My Drive/NLP_Outputs/out_multi.csv'

In [70]:
# with open(path_out_single, 'w') as f:
#   writer = csv.writer(f)
#   for idx in range(len(out_ensemble)):
#     writer.writerow([single_ids[idx], float(out_ensemble[idx])])

# with open(path_out_multi, 'w') as f:
#   writer = csv.writer(f)
#   for idx in range(len(out_ensemble_multi)):
#     writer.writerow([multi_ids[idx], float(out_ensemble_multi[idx])])